In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt


import pandas as pd
from sklearn.preprocessing import OneHotEncoder


# Explore Data

#### Train data

In [4]:
train = pd.read_csv('./Data/kaggle/train.csv')
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
train['date'].str[0:4].value_counts()

date
2016    650430
2013    648648
2014    648648
2015    648648
2017    404514
Name: count, dtype: int64

In [6]:
train['date'] = pd.to_datetime(train['date'])

In [7]:
train['store_nbr'].nunique(), train['family'].nunique()

(54, 33)

In [8]:
train['store_nbr'].value_counts().sort_index()

store_nbr
1     55572
2     55572
3     55572
4     55572
5     55572
6     55572
7     55572
8     55572
9     55572
10    55572
11    55572
12    55572
13    55572
14    55572
15    55572
16    55572
17    55572
18    55572
19    55572
20    55572
21    55572
22    55572
23    55572
24    55572
25    55572
26    55572
27    55572
28    55572
29    55572
30    55572
31    55572
32    55572
33    55572
34    55572
35    55572
36    55572
37    55572
38    55572
39    55572
40    55572
41    55572
42    55572
43    55572
44    55572
45    55572
46    55572
47    55572
48    55572
49    55572
50    55572
51    55572
52    55572
53    55572
54    55572
Name: count, dtype: int64

- Stores have the same number of records

In [9]:
train[train['store_nbr'] == 9]['family'].value_counts()

family
AUTOMOTIVE                    1684
HOME APPLIANCES               1684
SCHOOL AND OFFICE SUPPLIES    1684
PRODUCE                       1684
PREPARED FOODS                1684
POULTRY                       1684
PLAYERS AND ELECTRONICS       1684
PET SUPPLIES                  1684
PERSONAL CARE                 1684
MEATS                         1684
MAGAZINES                     1684
LIQUOR,WINE,BEER              1684
LINGERIE                      1684
LAWN AND GARDEN               1684
LADIESWEAR                    1684
HOME CARE                     1684
HOME AND KITCHEN II           1684
BABY CARE                     1684
HOME AND KITCHEN I            1684
HARDWARE                      1684
GROCERY II                    1684
GROCERY I                     1684
FROZEN FOODS                  1684
EGGS                          1684
DELI                          1684
DAIRY                         1684
CLEANING                      1684
CELEBRATION                   1684
BREAD/BAKERY 

- The number of items in each family in a store is the same

In [10]:
train[(train['store_nbr'] == 9) & (train['family'] == 'HARDWARE')]['date']

1763      2013-01-01
3545      2013-01-02
5327      2013-01-03
7109      2013-01-04
8891      2013-01-05
             ...    
2993741   2017-08-11
2995523   2017-08-12
2997305   2017-08-13
2999087   2017-08-14
3000869   2017-08-15
Name: date, Length: 1684, dtype: datetime64[ns]

In [11]:
# Generate a complete date range based on the minimum and maximum dates in the dataset
min_date = train['date'].min()
max_date = train['date'].max()
complete_date_range = pd.date_range(start=min_date, end=max_date, freq='D')

# Identify missing dates
missing_dates = complete_date_range.difference(train['date'])
missing_dates


DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)

- There are some missing dates into the dataset

#### Stores

In [12]:
stores = pd.read_csv('./Data/kaggle/stores.csv')
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [13]:
stores['city'].value_counts()

city
Quito            18
Guayaquil         8
Cuenca            3
Santo Domingo     3
Manta             2
Latacunga         2
Machala           2
Ambato            2
Quevedo           1
Esmeraldas        1
Loja              1
Libertad          1
Playas            1
Daule             1
Babahoyo          1
Salinas           1
Puyo              1
Guaranda          1
Ibarra            1
Riobamba          1
Cayambe           1
El Carmen         1
Name: count, dtype: int64

In [14]:
stores['state'].value_counts()

state
Pichincha                         19
Guayas                            11
Santo Domingo de los Tsachilas     3
Azuay                              3
Manabi                             3
Cotopaxi                           2
Tungurahua                         2
Los Rios                           2
El Oro                             2
Chimborazo                         1
Imbabura                           1
Bolivar                            1
Pastaza                            1
Santa Elena                        1
Loja                               1
Esmeraldas                         1
Name: count, dtype: int64

In [15]:
stores['type'].value_counts()

type
D    18
C    15
A     9
B     8
E     4
Name: count, dtype: int64

#### OIL

In [16]:
oil = pd.read_csv('./Data/kaggle/oil.csv')
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


#### Holidays Events

In [17]:
holidays_events = pd.read_csv('./Data/kaggle/holidays_events.csv')
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


#### Transcations

In [18]:
transactions = pd.read_csv('./Data/kaggle/transactions.csv')
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


#### Submission set

In [19]:
submission_df = pd.read_csv('./Data/kaggle/test.csv')

# Split in Train and Test

In [23]:
df_train = train.loc[train['date'].dt.year < 2017]
df_val = train.loc[train['date'].dt.year >= 2017]

In [24]:
df_train.to_csv('./Data/split/traning_set.csv', index=False)
df_val.to_csv('./Data/split/validation_set.csv', index=False)